# Crop Type Classification: CART

Our aim in this notebook is to classify crop type using PlanetScope 4-band Orthotiles. The crop types of particular interest are corn and soybeans.

[CART](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) is a decision tree algorithm that has shown great promise for classification of remotely sensed imagery. We will use this algorithm to classify crop type.

In this notebook, we will focus on using the PlanetScope imagery 4 bands as well as NDVI calculation as the features that are fed into the CART algorithm. We will train on one PS Orthotile and validate on another PS Orthotile.

### Outline
1. Identify datasets (PS Orthotiles and gold standard dataset for train and test)
1. Train classifier
1. Test classifier


### Gold Standard Dataset

The [USDA 2016 Crop Data Layer](https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php) (CDL) provides a national crop type dataset. This dataset was build using Landsat 8, DMC Deimos-1, and UK 2 satellite imagery ([src](https://www.nass.usda.gov/Research_and_Science/Cropland/sarsfaqs2.php#Section3_17.0)), using supervised classification (decision trees) based on ground truth from the Farm Service Agency (FSA) Common Land Unit (CLU) Program. Since this is a derived dataset, it isn't a ground truth dataset but it is known to be quite accurate so can be used as our gold standard dataset. This dataset is provided as a georegistered raster (geoTIFF).

Since corn and soybeans are the primary crops grown in Iowa, we will focus our analysis in that state. The [metadata](https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_ia16.htm) provided for the Iowa 2016 CDL indicates that it's accuracy is 96.4% and that corn (categorization code 1) and soybeans (categorization code 5) are indeed the primary crop types in the state.

## Identify Datasets


### 2016 Iowa CDL

Download data for Iowa from [USDA 2016 Crop Data Layer](https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php) (CDL). Filter to corn / soybeans. 

This is achieved by using the [CropScape](https://nassgeodata.gmu.edu/CropScape/) site.